In [306]:
import numpy as np
import pandas as pd
import sympy as sp
from sympy import *
import math
import os

In [307]:
#Input Pressure and Temperature values
P = float(input("Enter pressure in psia:")) # psia
T = float(input("Enter Temperature in Farenheit: ")) # Farenheit

print(f'For Pressure: {P} psia  and Temperature: {T} Farenheit' )

For Pressure: 1000.0 psia  and Temperature: -55.0 Farenheit


In [308]:
#Import Components Properties
component_properties = pd.read_excel("components.xlsx")
component_properties

,component,composition,Pci,Tci,wi,Mwi,Vci
0,N2,0.0200,493.1,227.49,0.040,28.013,0.0510
1,C1,0.8866,666.4,343.33,0.008,16.043,0.0988
2,C2,0.0492,706.5,549.92,0.098,30.070,0.0783
3,C3,0.0246,616.0,666.06,0.152,44.097,0.0727
4,nC4,0.0098,550.6,765.62,0.193,58.123,0.0703
5,nC5,0.0098,488.6,845.80,0.251,72.151,0.0675


In [309]:
#Import binary coefficients
d= pd.read_excel("binary.xlsx").to_numpy()
d

array([[0.    , 0.018 , 0.039 , 0.046 , 0.047 , 0.048 ],
       [0.018 , 0.    , 0.005 , 0.01  , 0.0145, 0.0182],
       [0.039 , 0.005 , 0.    , 0.0017, 0.0032, 0.0048],
       [0.046 , 0.01  , 0.0017, 0.    , 0.0012, 0.0024],
       [0.047 , 0.0145, 0.0032, 0.0012, 0.    , 0.0008],
       [0.048 , 0.0182, 0.0048, 0.0024, 0.0008, 0.    ]])

In [310]:
Pci = component_properties['Pci'] 
Tci = component_properties['Tci']

Pri= P/Pci
Tri= (T+459.67 )/Tci

print('Reduced Pressure,  Pri:\n', Pri)
print('Reduced Temperature, Tri:\n',Tri)

Reduced Pressure,  Pri:
 0    2.027986
1    1.500600
2    1.415428
3    1.623377
4    1.816201
5    2.046664
Name: Pci, dtype: float64
Reduced Temperature, Tri:
 0    1.778847
1    1.178662
2    0.735871
3    0.607558
4    0.528552
5    0.478446
Name: Tci, dtype: float64


In [311]:
wi = component_properties['wi'] #accentric factor

w_a = 0.45723553
w_b = 0.07779607

for i in wi:
    if i<0.49:
        mi = 0.3796+1.54226*wi-0.2699*wi**2
    else:
        mi = 0.379642+1.48503*wi-0.164423*wi**2+0.016667*wi**3
        
Ai =(w_a*(1+mi*(1-Tri**0.5))**2*(Pri/Tri**2)).replace(np.nan,0).to_numpy()
Bi = ((w_b*Pri)/Tri).to_numpy()
print('Ai:',Ai)
print('Bi:',Bi)

Ai: [0.21315405 0.46128023 1.38137769 2.58607939 4.15396658 6.19628096]
Bi: [0.0886919  0.0990452  0.14963872 0.20786879 0.26732139 0.33279046]


In [312]:
#Calculating A and B

c = component_properties['composition'].to_numpy()
n= len(c)

B =  np.sum(c * Bi)

A=0
for i in range (n):
    for j in range (n):
        A += (1-d[i,j])*(Ai[i]*Ai[j])**0.5*c[i]*c[j] 
        
print('A :', A)
print('B :', B)

A : 0.5651691201327123
B : 0.107944202075142


In [313]:
#Calculating Z 

m1 = 1+2**0.5
m2 = 1-2**0.5
p=1
q=(m1+m2-1)*B-1
r=(A+m1*m2*B**2-(m1+m2)*B*(B+1))
s=-A*B-m1*m2*B**2*(B+1)

Z = np.roots([p, q, r, s])

print(f'For Pressure: {P} psia  and Temperature: {T} Farenheit' )
print('                          A:', round(A,4))
print('                          B:', round(B,4))
print('                          Z:', Z)
for i in range(len(Z)):
    if np.isreal(Z[i]):
        print('Without imaginery values, Z:', np.real(Z[i]))
        
print('                      Z_max:', max(Z))
print('                      Z_min:', min(Z))

For Pressure: 1000.0 psia  and Temperature: -55.0 Farenheit
                          A: 0.5652
                          B: 0.1079
                          Z: [0.41292058+0.j         0.23956761+0.24307906j 0.23956761-0.24307906j]
Without imaginery values, Z: 0.4129205765436229
                      Z_max: (0.4129205765436229+0j)
                      Z_min: (0.23956761069061733-0.24307906481308494j)


# Wilsons Correlation , K-Values 

In [314]:

Ki = (1/Pri)*((5.37*(1+wi)*(1-(1/Tri))).apply(math.exp))

Ki 

0    5.687073
1    1.513815
2    0.085111
3    0.011329
4    0.001816
5    0.000323
dtype: float64

# Composition of Liquid and Vapour phase 

In [315]:
g_V = lambda v:sum(c*(Ki - 1)/(v*(Ki - 1)+1))
dg_V = lambda v:-sum(c*((Ki-1)**2)/((v*(Ki-1) + 1)**2))

A = sum(c*(Ki-1))
B = sum(c*(Ki-1)/Ki)

v = A/(A - B) 

while abs(g_V(v))>0.0001:
    v = v - g_V(v)/dg_V(v)

l = 1 - v

print(f"Composition in vapor phase is: {v}")
print(f"Composition in liquid phase is: {l}")



Composition in vapor phase is: 0.7754182515439129
Composition in liquid phase is: 0.22458174845608714


# Calculation of A and B in liquid and vapour phase

In [316]:
Xi = c/(1 + v*(Ki-1))
Yi = Xi*Ki

def Aa(c):
    return [sum(c[i]*c*(Ai[i]*Ai)**0.5) for i in range(len(c))]

Al = sum(Aa(Xi))
Av = sum(Aa(Yi))
Bl = sum(Xi*Bi)
Bv = sum(Yi*Bi)

print(f"A in Vapour phase {Av} and liquid phase {Al}")
print(f"B in Vapour phase {Bv} and liquid phase {Bl}")


A in Vapour phase 0.4654376987788452 and liquid phase 0.9959953217004294
B in Vapour phase 0.0996666779058611 and liquid phase 0.13652418848357134


In [317]:

# m1 = 1+2**0.5
# m2 = 1-2**0.5

# #Vapour Phase 
# p=1
# q=(m1+m2-1)*Bv-1
# r=(Av+m1*m2*Bv**2-(m1+m2)*Bv*(Bv+1))
# s=-Av*Bv-m1*m2*Bv**2*(Bv+1)

# Zv = np.roots([p, q, r, s])

# print('Vapour Phase compressibility factor, Zv:', Zv)

# Zvm = max(Zv)
# print('Vapour Phase :', Zvm)

# #Liquid phase 
# t=1
# u=(m1+m2-1)*Bl-1
# w=(Al+m1*m2*Bl**2-(m1+m2)*Bl*(Bl+1))
# x=-Al*Bl-m1*m2*Bl**2*(Bl+1)

# Zl = np.roots([t, u, w, x])

# for i in range(len(Zl)):
#     if np.isreal(Zl[i]):
#         print('Liquid Phase compressibility factor, Zl:', np.real(Zl[i]))




# Compresibility of Liquid and Vapour Phase #

In [318]:
Zl, Zv = symbols("Zl Zv")
f_Zl = Zl**3+(Bl-1)*Zl**2+(Al-3*(Bl)**2-2*Bl)*Zl-(Al*Bl-(Bl)**2-(Bl)**3)
f_Zv = Zv**3+(Bv-1)*Zv**2+(Av-3*(Bv)**2-2*Bv)*Zv-(Av*Bv-(Bv)**2-(Bv)**3)
Zl = solve(f_Zl,Zl)
Zv = solve(f_Zv,Zv)

# eliminate complex roots:
Zl = min([i for i in Zl if sp.im(i)==0]) # compressibility factor of liquid phase
Zv = max([i for i in Zv if sp.im(i)==0]) # compressibility factor of vapour phase
print(f"Vapour Phase compressibility factor, Zv: {Zv}")
print(f"Liquid Phase compressibility factor, Zl: {Zl}")

Vapour Phase compressibility factor, Zv: 0.607429599405608
Liquid Phase compressibility factor, Zl: 0.218131410065024


# Fugacity coefficients  and K-values correction # 

In [319]:
#fugacity of vapour phase  
phiv=(-math.log(Zv-Bv)+(Av/((m1-m2)*Bv)*(2*Ai*Yi/Av)-(Bi/Bv))*log(((Zv+m2*Bv)/(Zv+m1*Bv)))+Bi/Bv*(Zv-1)).apply(math.exp)

#fugacity of liquid phase  
phil=(-math.log(Zl-Bl)+(Al/((m1-m2)*Bl)*(2*Ai*Xi/Al)-(Bi/Bl))*log(((Zl+m2*Bl)/(Zl+m1*Bl)))+Bi/Bl*(Zl-1)).apply(math.exp)


# Equilibrium ratio:
K_i = (phil/phiv)

# print(f"fugacity co-eff of Vapour : {phiv}")
# print(f"fugacity co-eff of Liquid : {phil}")




## Equilibrium ratio anotherway
fv=phiv*Yi*P
fl=phil*Xi*P
K_ii = (Yi/Xi)*(fl/fv)


e_c=sum(((fl/fv)-1)**2)
e=sum(((fl/fv)-1))

print(f"Previous , Ki : {Ki.to_numpy()}")
print(f"===================================================================================")
print(f"Corrected, K_i : {K_i.to_numpy()}")
print(f"Corrected, K_ii: {K_ii.to_numpy()}")

Previous , Ki : [5.68707303e+00 1.51381494e+00 8.51106708e-02 1.13287747e-02
 1.81618559e-03 3.22519011e-04]
Corrected, K_i : [8.2661221  4.73583883 2.38713685 2.13305781 4.56579145 3.16478848]
Corrected, K_ii: [8.2661221  4.73583883 2.38713685 2.13305781 4.56579145 3.16478848]


In [320]:
# iteration = 0.1
# K_i = Ki

# while iteration > 0.000000000000001:
#     Ki = K_i
#     g_V = lambda v:sum(c*(Ki - 1)/(v*(Ki - 1)+1))
#     dg_V = lambda v:-sum(c*((Ki-1)**2)/((v*(Ki-1) + 1)**2))

#     A = sum(c*(Ki-1))
#     B = sum(c*(Ki-1)/Ki)

#     v = A/(A - B) 

#     while abs(g_V(v))>0.0001:
#         v = v - g_V(v)/dg_V(v)
#     l = 1 - v
    
#     Xi = c/(1 + v*(Ki-1))
#     Yi = Xi*Ki

#     def Aa(c):
#         return [sum(c[i]*c*(Ai[i]*Ai)**0.5) for i in range(len(c))]

#     Al = sum(Aa(Xi))
#     Av = sum(Aa(Yi))
#     Bl = sum(Xi*Bi)
#     Bv = sum(Yi*Bi)
    
#     Zl, Zv = symbols("Zl Zv")
#     f_Zl = Zl**3+(Bl-1)*Zl**2+(Al-3*(Bl)**2-2*Bl)*Zl-(Al*Bl-(Bl)**2-(Bl)**3)
#     f_Zv = Zv**3+(Bv-1)*Zv**2+(Av-3*(Bv)**2-2*Bv)*Zv-(Av*Bv-(Bv)**2-(Bv)**3)
#     Zl = solve(f_Zl,Zl)
#     Zv = solve(f_Zv,Zv)

#     # eliminate complex roots:
#     Zl = min([i for i in Zl if sp.im(i)==0]) 
#     Zv = max([i for i in Zv if sp.im(i)==0])

#     phiv=(-math.log(Zv-Bv)+(Av/((m1-m2)*Bv)*(2*Ai*Yi/Av)-(Bi/Bv))*log(((Zv+m2*Bv)/(Zv+m1*Bv)))+Bi/Bv*(Zv-1)).apply(math.exp)
#     phil=(-math.log(Zl-Bl)+(Al/((m1-m2)*Bl)*(2*Ai*Xi/Al)-(Bi/Bl))*log(((Zl+m2*Bl)/(Zl+m1*Bl)))+Bi/Bl*(Zl-1)).apply(math.exp)

#     # Equilibrium ratio:
#     K_i = (phil/phiv)
#     iteration=sum(((fl/fv)-1)**2)
# iteration


# Density Calculation 

In [350]:
R= 10.731
Tk = T + 459.67 #in Rankine
Mwi= component_properties.Mwi
Mwl = sum(Xi*Mwi) # apparent molecular weight of liquid phase
Mwv = sum(Yi*Mwi) # apparent molecular weight of vapour phase
dl=(P*Mwl)/(R*Tk*Zl) #liquid density
dv=(P*Mwv)/(R*Tk*Zv) #vapor density

print(f"Density of Liquid : {dl}")
print(f"Density of Vapour : {dv}")


Density of Liquid : 0    0.127624119066510
1     10.7378124313442
2     5.37497212426476
3     4.90734537165148
4     2.66088046366960
5     3.32010037094263
dtype: object
Density of Vapour : 6.28428940145513


# Viscosity Calculation

In [348]:
for i in range (n):
    Vpc = sum(component_properties.Vci* Xi)
    Tpc = sum(Xi* Tci)
    Ppc =sum(Xi*Pci)

    Ei=5.4402*Tci**(1/6)/(Mwl**0.5*Pci**(2/3))

    if Tri[i]<=1.5:
        usi=34*10**(-5)*(Tri)**0.94/Ei
    elif Tri[i]>1.5:
        usi=(17.78*10**(-5)*(4.58*Tri-1.67)**0.625)/Ei
    us1 =Xi*usi*(Mwi**0.5)
    us2 =Xi*(Mwi**0.5)
    us = us1/us2


dr=Vpc*(dl/Mwl)
xv=3.5+986/Tk+0.01*Mwv
yv=2.4-0.2*xv
kv=(9.4+0.02*Mwv)*(Tk**1.5)/(209+19*Mwv+Tk)
Em=5.4402*Tpc**(1/6)/(Mwl**0.5*Ppc**(2/3))

#viscosity of vapour
visv=10**(-4)*kv*math.exp(xv*(dr/62.4)**yv)
print(f"Viscosity of Vapour : {visv}")

# viscosity of liquid
visl=(us)+(1/Em)*((0.1023+0.023364*dr+0.058533*dr**2-0.040758*dr**3+0.0093724*dr**4)**4-1*10**(-4)) 
print(f"Viscosity of Liquid : {visl}")


Viscosity of Vapour : 0.00855555368634406
Viscosity of Liquid : 0     0.0143001952749530
1     0.0112077573670412
2    0.00712807932894284
3    0.00540570811107116
4    0.00441131490622743
5    0.00374307605561677
dtype: object


In [355]:
result = pd.DataFrame({"Component":component_properties.component,
                        "Molecular Weight":component_properties.Mwi,
                        "Composition [Given]":component_properties.composition,
                        "Mw of in Liquid": Xi*Mwi,
                        "Mw of in Vapor": Yi*Mwi,
                        "Composition in Liquid":round(Xi,4),
                        "Composition in Vapour":round(Yi,4), 
                        "Equilibrium Ratio":round(K_i,4)})
result

,Component,Molecular Weight,Composition [Given],Mw of in Liquid,Mw of in Vapor,Composition in Liquid,Composition in Vapour,Equilibrium Ratio
0,N2,28.013,0.0200,0.120890,0.687513,0.0043,0.0245,8.2661
1,C1,16.043,0.8866,10.171271,15.397422,0.6340,0.9598,4.7358
2,C2,30.070,0.0492,5.091381,0.433331,0.1693,0.0144,2.3871
3,C3,44.097,0.0246,4.648427,0.052661,0.1054,0.0012,2.1331
4,nC4,58.123,0.0098,2.520489,0.004578,0.0434,0.0001,4.5658
5,nC5,72.151,0.0098,3.144927,0.001014,0.0436,0.0000,3.1648
